In [ ]:
# Google Colabでライブラリをアップロードする
from google.colab import files
uploaded = files.upload()

In [ ]:
# Google Colabでドライブのデータを使う
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# Google Colabでライブラリをインストールする
!pip install japanize_matplotlib bottleneck tslearn

In [1]:
# 自作関数
import MasterResearchFunction as mr

# 基本ライブラリ
import csv
import math
import os
import pickle
import random
import re
import statistics
from datetime import datetime, timedelta
from decimal import Decimal

# 数値計算とデータ処理
import bottleneck as bn
import numpy as np
import pandas as pd

# 機械学習ライブラリ
from sklearn import preprocessing
from sklearn.metrics import accuracy_score, classification_report
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler

# ディープラーニングライブラリ
import tensorflow as tf
from tensorflow.keras.layers import (
    Activation,
    Add,
    Conv1D,
    Dense,
    Dropout,
    Flatten,
    GlobalAveragePooling1D,
    Input,
    LayerNormalization,
    LSTM,
    Masking,
    MaxPooling1D,
    MultiHeadAttention,
)
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical


# プロットと可視化
import japanize_matplotlib
import matplotlib.dates as mdates
import matplotlib.pyplot as plt

# その他のライブラリ
from fastdtw import fastdtw
from scipy import signal, stats
from scipy.interpolate import interp1d, UnivariateSpline
from scipy.signal import savgol_filter
from scipy.spatial.distance import euclidean
from tslearn.metrics import dtw_path

In [ ]:
motion_data = mr.process_apple_watch_csv('/Users/hinase/CodeChord/MasterResearch/datasets/new/nakazawa/motion/raw/nakazawa_circle_game.csv')
eye_data = mr.process_tobii_csv('/Users/hinase/CodeChord/MasterResearch/datasets/new/nakazawa/eye/raw/nakazawa5_circle_walk.csv')

In [ ]:
# 25秒減算坂本
#eye_data["Timestamp"] = eye_data["Timestamp"] - pd.to_timedelta(25.659, unit='s')
#eye_data["Timestamp"] = eye_data["Timestamp"] - pd.to_timedelta(25.4841, unit='s')
#eye_data["Timestamp"] = eye_data["Timestamp"] - pd.to_timedelta(22.831, unit='s')
# 29秒減算渡部
#eye_data["Timestamp"] = eye_data["Timestamp"] - pd.to_timedelta(29.644, unit='s')
# 43秒減算小寺
#eye_data["Timestamp"] = eye_data["Timestamp"] - pd.to_timedelta(43.0029, unit='s')
# 43秒減算小寺(tri, check)
#eye_data["Timestamp"] = eye_data["Timestamp"] - pd.to_timedelta(42.045, unit='s')
#eye_data["Timestamp"] = eye_data["Timestamp"] - pd.to_timedelta(42.958, unit='s')
# 43秒減算中沢（circle, tri）
#eye_data["Timestamp"] = eye_data["Timestamp"] - pd.to_timedelta(38.43, unit='s')
# 43秒減算中沢（cross）
#eye_data["Timestamp"] = eye_data["Timestamp"] + pd.to_timedelta(11.486, unit='s')
#eye_data["Timestamp"] = eye_data["Timestamp"] - pd.to_timedelta(43.66, unit='s')
# 43秒減算水野（cross）
#eye_data["Timestamp"] = eye_data["Timestamp"] - pd.to_timedelta(2.043, unit='s')
# 43秒減算水野（tri）
#eye_data["Timestamp"] = eye_data["Timestamp"] - pd.to_timedelta(2.171, unit='s')
# 43秒減算水野（circle, check）
#eye_data["Timestamp"] = eye_data["Timestamp"] - pd.to_timedelta(1.994, unit='s')
#eye_data["Timestamp"] = eye_data["Timestamp"] - pd.to_timedelta(0.1, unit='s')
# 中沢
eye_data["Timestamp"] = eye_data["Timestamp"] - pd.to_timedelta(1.1, unit='s')

In [ ]:
eye_data["Timestamp"]

In [ ]:
motion_data["Timestamp"]

In [ ]:
Timestamp('2024-11-08 14:45:25.335983038')

In [ ]:
#hinase_circle_seg = mr.three_axis_spring(motion_data, train_data, [5, 5, 5], 'acc')

In [ ]:
Hz = 100

In [ ]:
a = mr.find_true_intervals(motion_data)

In [ ]:
a

In [ ]:
motion_data["Timestamp"][4813]

In [ ]:
#b = []

In [ ]:
print(i)

In [ ]:
i = i + 1

In [ ]:
i

In [ ]:
a[i]

In [ ]:
b.append([a[i][0]+80, a[i][1]-130])

In [ ]:
b

In [ ]:
len(b)

In [ ]:
del b[11]

In [ ]:
for j in range(len(b)):
  print(b[j][1] - b[j][0])

In [ ]:
for i in range(len(a)):
  print(a[i][1] - a[i][0])

In [ ]:
len(b)

In [ ]:
del b[10]

In [ ]:
c = []
for i in range(len(b)):
  c.append([motion_data["Timestamp"][b[i][0]], motion_data["Timestamp"][b[i][1]]])

In [ ]:
c

In [ ]:
# 各範囲内のデータを格納するリストを用意します
result = []
# cの各行に対してループを回します
for start_time, end_time in extracted_data:
    # 範囲内のデータを抽出するためのマスクを作成します
    mask = (eye_data["Timestamp"] >= start_time) & (eye_data["Timestamp"] <= end_time)
    # マスクを適用してデータを抽出します
    data_in_range = eye_data.loc[mask]
    # 結果をリストに追加します
    result.append(data_in_range)

# 結果を確認します
for idx, data in enumerate(result):
    print(f"範囲 {idx+1}:")
    print(data)
    print("\n")

In [ ]:
# 定数の定義

LABELS = ['circle', 'cross', 'tri', 'check']
COLUMNS_TO_DROP = [
    'Sensor', 'Participant name', 'Event', 'Event value',
    'Eye movement type', 'Eye movement type index', 'Ungrouped',
    'Validity left', 'Validity right', 'Gaze event duration', 'Gaze2D_Distance',
    'Fixation_Distance', 'Gaze3D_Distance', 'Pupil_Diameter_Change',
    'GazeDirection_Distance', 'PupilPosition_Distance'
]
# インターバルのパラメータ設定
MIN_INTERVAL = 0.84  # 最小インターバル長（秒）
MAX_INTERVAL = 3.04  # 最大インターバル長（秒）
NUM_INTERVALS = 160  # 抽出するインターバルの数

In [6]:
# 既に読み込んだモーションデータを使用
motion_data = motion_routine_data

In [9]:
a = mr.find_true_intervals(motion_data)

In [11]:
motion_data["Timestamp"][93442]

Timestamp('2024-10-21 17:04:20.321764946')

In [3]:
# データとラベルを格納するリストの初期化
X_filled = []
y_labels = []
true_labels = []
# データとラベルを格納する辞書の初期化
data_dict = {}
label_dict = {}
true_label_dict = {}

for name in NAMES:
    print(f"Processing {name}...")

    # データの読み込みと前処理
    eye_data, gesture_labels = mr.load_gesture_eye_data_pickle(name, LABELS, COLUMNS_TO_DROP)
    # 変数に保存
    data_dict[name] = {
        'gesture': eye_data,
        'non_gesture': []
    }
    label_dict[name] = {
        'gesture': gesture_labels,
        'non_gesture': []
    }
    true_label_dict[name] = {
        'gesture': ['gesture'] * len(eye_data),
        'non_gesture': []
    }

    # モーションデータとアイデータの読み込みと前処理
    motion_routine_data, eye_routine_data = mr.load_motion_and_eye_data(name)
    # 変数に保存
    motion_data = motion_routine_data

    # 除外インターバルの取得
    exclusion_intervals = mr.get_exclusion_intervals(motion_data)

    # データ全体の時間範囲の取得
    start_time = motion_routine_data['Timestamp'].min()
    end_time = motion_routine_data['Timestamp'].max()

    # 利用可能なインターバルの取得
    available_intervals = mr.get_available_intervals(exclusion_intervals, start_time, end_time)

    print(f"{name} の利用可能なインターバル数: {len(available_intervals)}")

    # ランダムなインターバルの抽出（eye_routine_data を使用）
    extracted_data = mr.extract_random_intervals(
        eye_routine_data, available_intervals, MIN_INTERVAL, MAX_INTERVAL, NUM_INTERVALS
    )

    # 抽出されたデータを辞書に保存
    data_dict[name]['non_gesture'] = extracted_data
    label_dict[name]['non_gesture'] = ['non_gesture'] * len(extracted_data)
    true_label_dict[name]['non_gesture'] = ['non_gesture'] * len(extracted_data)

    # ジェスチャーデータの統合
    for seq, lbl, true_lbl in zip(eye_data, gesture_labels, true_label_dict[name]['gesture']):
        if seq is not None and len(seq) > 0:
            X_filled.append(seq)
            y_labels.append(lbl)
            true_labels.append(true_lbl)
        else:
            print(f"{name} のジェスチャーシーケンスが空です。対応するラベルをスキップします。")

    # 非ジェスチャーデータの統合
    for seq, lbl, true_lbl in zip(data_dict[name]['non_gesture'], label_dict[name]['non_gesture'], true_label_dict[name]['non_gesture']):
        if isinstance(seq, pd.DataFrame):
            if not seq.empty:
                X_filled.append(seq)
                y_labels.append(lbl)
                true_labels.append(true_lbl)
            else:
                print(f"{name} の非ジェスチャーシーケンスが空です。対応するラベルをスキップします。")
        else:
            print(f"{name} の非ジェスチャーシーケンスが DataFrame ではありません。スキップします。")

print(f"総データ数: {len(X_filled)}")
print(f"総ラベル数（y_labels）: {len(y_labels)}")
print(f"総真偽ラベル数（true_labels）: {len(true_labels)}")


Processing sakamoto...
Exclusion Interval 0: start=2024-10-03 14:40:50.613534927, end=2024-10-03 14:40:54.043292999, types=(<class 'pandas._libs.tslibs.timestamps.Timestamp'>, <class 'pandas._libs.tslibs.timestamps.Timestamp'>)
Exclusion Interval 1: start=2024-10-03 14:40:58.009015083, end=2024-10-03 14:41:01.316668034, types=(<class 'pandas._libs.tslibs.timestamps.Timestamp'>, <class 'pandas._libs.tslibs.timestamps.Timestamp'>)
Exclusion Interval 2: start=2024-10-03 14:41:03.965172052, end=2024-10-03 14:41:07.862525940, types=(<class 'pandas._libs.tslibs.timestamps.Timestamp'>, <class 'pandas._libs.tslibs.timestamps.Timestamp'>)
Exclusion Interval 3: start=2024-10-03 14:41:10.331334114, end=2024-10-03 14:41:14.558215141, types=(<class 'pandas._libs.tslibs.timestamps.Timestamp'>, <class 'pandas._libs.tslibs.timestamps.Timestamp'>)
Exclusion Interval 4: start=2024-10-03 14:41:17.156810045, end=2024-10-03 14:41:21.034349918, types=(<class 'pandas._libs.tslibs.timestamps.Timestamp'>, <cla

In [33]:
X_filled[0]

,Gaze point X,Gaze point Y,Gaze point 3D X,Gaze point 3D Y,Gaze point 3D Z,Gaze direction left X,Gaze direction left Y,Gaze direction left Z,Gaze direction right X,Gaze direction right Y,...,Pupil position left Y,Pupil position left Z,Pupil position right X,Pupil position right Y,Pupil position right Z,Pupil diameter left,Pupil diameter right,Fixation point X,Fixation point Y,Timestamp
1755,929.0,452.0,-0.8099,49.8865,386.2720,-0.06853,0.17066,0.98294,0.06864,0.20717,...,-32.20,-30.57,-32.37,-28.15,-27.39,4.592,4.509,894.0,524.0,2024-10-03 13:56:10.029568
1756,970.0,378.0,-15.2079,77.2567,397.0173,-0.09811,0.22547,0.96930,0.03397,0.26031,...,-31.77,-30.89,-32.51,-27.75,-27.73,4.637,4.448,894.0,524.0,2024-10-03 13:56:10.049517
1757,943.0,321.0,-6.2108,99.0907,405.8049,-0.05259,0.25112,0.96653,0.02971,0.31366,...,-31.31,-31.17,-32.53,-27.11,-28.12,4.100,4.526,NaN,NaN,2024-10-03 13:56:10.069506
1758,956.0,303.0,-16.2740,162.1313,622.7105,-0.04435,0.25907,0.96484,-0.00090,0.30099,...,-30.83,-31.45,-32.36,-26.05,-29.09,3.972,4.695,NaN,NaN,2024-10-03 13:56:10.089518
1759,983.0,297.0,-31.0319,165.2716,622.2255,NaN,NaN,NaN,0.00169,0.28190,...,NaN,NaN,-32.18,-26.03,-28.87,NaN,4.115,NaN,NaN,2024-10-03 13:56:10.109505
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1845,732.0,653.0,72.7576,-19.6692,424.9909,0.08985,0.02230,0.99571,0.22439,0.02595,...,-32.96,-30.68,-31.52,-28.63,-27.37,4.569,4.642,736.0,659.0,2024-10-03 13:56:11.828497
1846,735.0,632.0,72.3599,-12.2712,429.9072,0.08783,0.04110,0.99529,0.22153,0.03823,...,-32.90,-30.56,-31.57,-28.59,-27.39,4.544,4.657,736.0,659.0,2024-10-03 13:56:11.848472
1847,739.0,619.0,71.6873,-7.5237,435.9109,0.08530,0.04994,0.99510,0.21743,0.04831,...,-32.86,-30.50,-31.62,-28.55,-27.40,4.518,4.668,736.0,659.0,2024-10-03 13:56:11.868462
1848,746.0,606.0,72.3918,-2.5831,455.9818,0.08332,0.05635,0.99493,0.21006,0.05779,...,-32.83,-30.48,-31.66,-28.50,-27.37,4.505,4.635,736.0,659.0,2024-10-03 13:56:11.888442


In [4]:
# タイムスタンプのギャップを確認
for name in NAMES:
    non_gesture_intervals = data_dict[name]['non_gesture']
    if non_gesture_intervals:
        for idx, eye_routine_data in enumerate(non_gesture_intervals):
            if isinstance(eye_routine_data, pd.DataFrame):
                if not eye_routine_data.empty:
                    time_diffs = eye_routine_data['Timestamp'].diff().dt.total_seconds()
                    large_gaps = time_diffs[time_diffs > 0.05]
                    print(f"{name} の non_gesture_intervals[{idx}] の大きなタイムスタンプギャップ数: {len(large_gaps)}")
                else:
                    print(f"{name} の non_gesture_intervals[{idx}] が空です。")
            else:
                print(f"{name} の non_gesture_intervals[{idx}] が DataFrame ではありません。")

sakamoto の non_gesture_intervals[0] の大きなタイムスタンプギャップ数: 0
sakamoto の non_gesture_intervals[1] の大きなタイムスタンプギャップ数: 0
sakamoto の non_gesture_intervals[2] の大きなタイムスタンプギャップ数: 0
sakamoto の non_gesture_intervals[3] の大きなタイムスタンプギャップ数: 0
sakamoto の non_gesture_intervals[4] の大きなタイムスタンプギャップ数: 0
sakamoto の non_gesture_intervals[5] の大きなタイムスタンプギャップ数: 0
sakamoto の non_gesture_intervals[6] の大きなタイムスタンプギャップ数: 0
sakamoto の non_gesture_intervals[7] の大きなタイムスタンプギャップ数: 0
sakamoto の non_gesture_intervals[8] の大きなタイムスタンプギャップ数: 0
sakamoto の non_gesture_intervals[9] の大きなタイムスタンプギャップ数: 0
sakamoto の non_gesture_intervals[10] の大きなタイムスタンプギャップ数: 0
sakamoto の non_gesture_intervals[11] の大きなタイムスタンプギャップ数: 0
sakamoto の non_gesture_intervals[12] の大きなタイムスタンプギャップ数: 0
sakamoto の non_gesture_intervals[13] の大きなタイムスタンプギャップ数: 0
sakamoto の non_gesture_intervals[14] の大きなタイムスタンプギャップ数: 0
sakamoto の non_gesture_intervals[15] の大きなタイムスタンプギャップ数: 0
sakamoto の non_gesture_intervals[16] の大きなタイムスタンプギャップ数: 0
sakamoto の non_gesture_intervals[17] の大きな

In [5]:
true_labels

['gesture',
 'gesture',
 'gesture',
 'gesture',
 'gesture',
 'gesture',
 'gesture',
 'gesture',
 'gesture',
 'gesture',
 'gesture',
 'gesture',
 'gesture',
 'gesture',
 'gesture',
 'gesture',
 'gesture',
 'gesture',
 'gesture',
 'gesture',
 'gesture',
 'gesture',
 'gesture',
 'gesture',
 'gesture',
 'gesture',
 'gesture',
 'gesture',
 'gesture',
 'gesture',
 'gesture',
 'gesture',
 'gesture',
 'gesture',
 'gesture',
 'gesture',
 'gesture',
 'gesture',
 'gesture',
 'gesture',
 'gesture',
 'gesture',
 'gesture',
 'gesture',
 'gesture',
 'gesture',
 'gesture',
 'gesture',
 'gesture',
 'gesture',
 'gesture',
 'gesture',
 'gesture',
 'gesture',
 'gesture',
 'gesture',
 'gesture',
 'gesture',
 'gesture',
 'gesture',
 'gesture',
 'gesture',
 'gesture',
 'gesture',
 'gesture',
 'gesture',
 'gesture',
 'gesture',
 'gesture',
 'gesture',
 'gesture',
 'gesture',
 'gesture',
 'gesture',
 'gesture',
 'gesture',
 'gesture',
 'gesture',
 'gesture',
 'gesture',
 'non_gesture',
 'non_gesture',
 'non_ge

In [6]:
X_filled[1000].columns

Index(['Gaze point X', 'Gaze point Y', 'Gaze point 3D X', 'Gaze point 3D Y',
       'Gaze point 3D Z', 'Gaze direction left X', 'Gaze direction left Y',
       'Gaze direction left Z', 'Gaze direction right X',
       'Gaze direction right Y', 'Gaze direction right Z',
       'Pupil position left X', 'Pupil position left Y',
       'Pupil position left Z', 'Pupil position right X',
       'Pupil position right Y', 'Pupil position right Z',
       'Pupil diameter left', 'Pupil diameter right', 'Fixation point X',
       'Fixation point Y', 'Timestamp'],
      dtype='object')

In [ ]:
# 2. 前処理とフィルタリングの適用
filtered_X, filtered_y, filtered_true = mr.preprocess_and_filter_sequences(
    X_filled,
    y_labels,
    true_labels,
    sampling_rate=50.0,
    max_gap_seconds=0.1,
    max_nan_seconds=1.0
)

/Users/hinase/CodeChord/MasterResearch/MasterResearchFunction.py:820: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df[gaze_col] = df[gaze_col].fillna(method='ffill').fillna(method='bfill')
/Users/hinase/CodeChord/MasterResearch/MasterResearchFunction.py:839: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df[gaze_col] = df[gaze_col].fillna(method='ffill').fillna(method='bfill')
/Users/hinase/CodeChord/MasterResearch/MasterResearchFunction.py:859: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df[spline_col] = df[spline_col].fillna(method='ffill').fillna(method='bfill')
/Users/hinase/CodeChord/MasterResearch/MasterResearchFunction.py:871: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future versi

シーケンス 215 は1秒以上連続するNaNを含むため、削除されました。


/Users/hinase/CodeChord/MasterResearch/MasterResearchFunction.py:820: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df[gaze_col] = df[gaze_col].fillna(method='ffill').fillna(method='bfill')
/Users/hinase/CodeChord/MasterResearch/MasterResearchFunction.py:839: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df[gaze_col] = df[gaze_col].fillna(method='ffill').fillna(method='bfill')
/Users/hinase/CodeChord/MasterResearch/MasterResearchFunction.py:859: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df[spline_col] = df[spline_col].fillna(method='ffill').fillna(method='bfill')
/Users/hinase/CodeChord/MasterResearch/MasterResearchFunction.py:871: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future versi

残る NaN を 0 で埋めます。
残る NaN を 0 で埋めます。
残る NaN を 0 で埋めます。
残る NaN を 0 で埋めます。
残る NaN を 0 で埋めます。
シーケンス 1085 は1秒以上連続するNaNを含むため、削除されました。
残る NaN を 0 で埋めます。


/Users/hinase/CodeChord/MasterResearch/MasterResearchFunction.py:820: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df[gaze_col] = df[gaze_col].fillna(method='ffill').fillna(method='bfill')
/Users/hinase/CodeChord/MasterResearch/MasterResearchFunction.py:839: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df[gaze_col] = df[gaze_col].fillna(method='ffill').fillna(method='bfill')
/Users/hinase/CodeChord/MasterResearch/MasterResearchFunction.py:859: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df[spline_col] = df[spline_col].fillna(method='ffill').fillna(method='bfill')
/Users/hinase/CodeChord/MasterResearch/MasterResearchFunction.py:871: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future versi

...

/Users/hinase/CodeChord/MasterResearch/MasterResearchFunction.py:871: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df[col] = df[col].fillna(method='ffill').fillna(method='bfill')
/Users/hinase/CodeChord/MasterResearch/MasterResearchFunction.py:820: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df[gaze_col] = df[gaze_col].fillna(method='ffill').fillna(method='bfill')
/Users/hinase/CodeChord/MasterResearch/MasterResearchFunction.py:839: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df[gaze_col] = df[gaze_col].fillna(method='ffill').fillna(method='bfill')
/Users/hinase/CodeChord/MasterResearch/MasterResearchFunction.py:859: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ff

残る NaN を 0 で埋めます。
残る NaN を 0 で埋めます。


/Users/hinase/CodeChord/MasterResearch/MasterResearchFunction.py:871: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df[col] = df[col].fillna(method='ffill').fillna(method='bfill')
/Users/hinase/CodeChord/MasterResearch/MasterResearchFunction.py:820: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df[gaze_col] = df[gaze_col].fillna(method='ffill').fillna(method='bfill')
/Users/hinase/CodeChord/MasterResearch/MasterResearchFunction.py:839: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df[gaze_col] = df[gaze_col].fillna(method='ffill').fillna(method='bfill')
/Users/hinase/CodeChord/MasterResearch/MasterResearchFunction.py:859: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ff

残る NaN を 0 で埋めます。
残る NaN を 0 で埋めます。
残る NaN を 0 で埋めます。
フィルタリング後のデータ数: 1219


/Users/hinase/CodeChord/MasterResearch/MasterResearchFunction.py:839: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df[gaze_col] = df[gaze_col].fillna(method='ffill').fillna(method='bfill')
/Users/hinase/CodeChord/MasterResearch/MasterResearchFunction.py:859: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df[spline_col] = df[spline_col].fillna(method='ffill').fillna(method='bfill')
/Users/hinase/CodeChord/MasterResearch/MasterResearchFunction.py:871: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df[col] = df[col].fillna(method='ffill').fillna(method='bfill')
/Users/hinase/CodeChord/MasterResearch/MasterResearchFunction.py:820: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use ob

In [11]:
X_scaled[0].columns

Index(['Gaze point X', 'Gaze point Y', 'Gaze point 3D X', 'Gaze point 3D Y',
       'Gaze point 3D Z', 'Gaze direction left X', 'Gaze direction left Y',
       'Gaze direction left Z', 'Gaze direction right X',
       'Gaze direction right Y', 'Gaze direction right Z',
       'Pupil position left X', 'Pupil position left Y',
       'Pupil position left Z', 'Pupil position right X',
       'Pupil position right Y', 'Pupil position right Z',
       'Pupil diameter left', 'Pupil diameter right', 'Fixation point X',
       'Fixation point Y', 'Fixation point X_Mask', 'Fixation point Y_Mask'],
      dtype='object')

In [18]:
filtered_true

['gesture',
 'gesture',
 'gesture',
 'gesture',
 'gesture',
 'gesture',
 'gesture',
 'gesture',
 'gesture',
 'gesture',
 'gesture',
 'gesture',
 'gesture',
 'gesture',
 'gesture',
 'gesture',
 'gesture',
 'gesture',
 'gesture',
 'gesture',
 'gesture',
 'gesture',
 'gesture',
 'gesture',
 'gesture',
 'gesture',
 'gesture',
 'gesture',
 'gesture',
 'gesture',
 'gesture',
 'gesture',
 'gesture',
 'gesture',
 'gesture',
 'gesture',
 'gesture',
 'gesture',
 'gesture',
 'gesture',
 'gesture',
 'gesture',
 'gesture',
 'gesture',
 'gesture',
 'gesture',
 'gesture',
 'gesture',
 'gesture',
 'gesture',
 'gesture',
 'gesture',
 'gesture',
 'gesture',
 'gesture',
 'gesture',
 'gesture',
 'gesture',
 'gesture',
 'gesture',
 'gesture',
 'gesture',
 'gesture',
 'gesture',
 'gesture',
 'gesture',
 'gesture',
 'gesture',
 'gesture',
 'gesture',
 'gesture',
 'gesture',
 'gesture',
 'gesture',
 'gesture',
 'gesture',
 'gesture',
 'gesture',
 'gesture',
 'gesture',
 'non_gesture',
 'non_gesture',
 'non_ge

In [12]:
# 最終的なデータの確認
print(f"処理後のデータ数: {len(X_scaled)}")
print(f"ラベル配列の長さ（y_labels_scaled）: {len(y_scaled)}")
print(f"真偽ラベル配列の長さ（true_labels_scaled）: {len(true_scaled)}")

処理後のデータ数: 1219
ラベル配列の長さ（y_labels_scaled）: 1219
真偽ラベル配列の長さ（true_labels_scaled）: 1219


In [17]:
X_scaled = []
scaler = StandardScaler()
for i in range(len(filtered_X)):
    X_scaled.append(scaler.fit_transform(filtered_X[i]))

In [19]:
# 1. ラベルのエンコーディング
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(filtered_true)
num_classes = len(label_encoder.classes_)
y_categorical = to_categorical(y_encoded, num_classes=num_classes)

# 2. データの分割（パディングの前に行う）
X_train_raw, X_test_raw, y_train, y_test = train_test_split(
    X_scaled, y_categorical, test_size=0.3, random_state=42
)

# 3. 訓練データとテストデータでそれぞれ最大シーケンス長を計算
max_length_train = max(len(seq) for seq in X_train_raw)
max_length_test = max(len(seq) for seq in X_test_raw)
max_length = max(max_length_train, max_length_test)

# 4. シーケンスのパディング
X_train_padded = pad_sequences(X_train_raw, maxlen=max_length, padding='post', value=0.0, dtype='float32')
X_test_padded = pad_sequences(X_test_raw, maxlen=max_length, padding='post', value=0.0, dtype='float32')

# 5. パディング後のデータに NaN が含まれていないか確認
print('NaN in X_train_padded:', np.isnan(X_train_padded).any())
print('NaN in X_test_padded:', np.isnan(X_test_padded).any())

# 必要に応じて NaN を 0 で置換
if np.isnan(X_train_padded).any():
    X_train_padded = np.nan_to_num(X_train_padded, nan=0.0)
if np.isnan(X_test_padded).any():
    X_test_padded = np.nan_to_num(X_test_padded, nan=0.0)

NaN in X_train_padded: False
NaN in X_test_padded: False


In [ ]:
# 3. データが存在するか確認
if not X_scaled:
    print("前処理後のデータが存在しません。")
else:
    # 4. データの分割（パディングの前に行う）
    X_train_raw, X_test_raw, y_train, y_test = train_test_split(
        X_scaled, true_scaled, test_size=0.3, random_state=42
    )

    # 5. 訓練データとテストデータでそれぞれ最大シーケンス長を計算
    max_length_train = max(len(seq) for seq in X_train_raw)
    max_length_test = max(len(seq) for seq in X_test_raw)
    max_length = max(max_length_train, max_length_test)

    # 6. シーケンスのパディング
    X_train_padded = pad_sequences(X_train_raw, maxlen=max_length, padding='post', value=0.0, dtype='float32')
    X_test_padded = pad_sequences(X_test_raw, maxlen=max_length, padding='post', value=0.0, dtype='float32')

    # 7. パディング後のデータに NaN が含まれていないか確認
    print('NaN in X_train_padded:', np.isnan(X_train_padded).any())
    print('NaN in X_test_padded:', np.isnan(X_test_padded).any())

    # 必要に応じて NaN を 0 で置換
    if np.isnan(X_train_padded).any():
        X_train_padded = np.nan_to_num(X_train_padded, nan=0.0)
    if np.isnan(X_test_padded).any():
        X_test_padded = np.nan_to_num(X_test_padded, nan=0.0)

    # 8. データの整合性確認
    print(f"処理後のデータ数: {len(X_scaled)}")
    print(f"ラベル配列の長さ（y_labels_scaled）: {len(y_scaled)}")
    print(f"真偽ラベル配列の長さ（true_labels_scaled）: {len(true_scaled)}")

    # 9. 機械学習の入力データとラベルの整形
    # 特徴ベクトルのリストを作成
    feature_vectors = [seq for seq in X_scaled if len(seq) > 0]


NaN in X_train_padded: False
NaN in X_test_padded: False
処理後のデータ数: 1218
ラベル配列の長さ（y_labels_scaled）: 1218
真偽ラベル配列の長さ（true_labels_scaled）: 1218


In [22]:
print(f'Number of samples in X_scaled: {len(X_scaled)}')
print(f'Number of samples in y_categorical: {len(true_scaled)}')

Number of samples in X_scaled: 1218
Number of samples in y_categorical: 1218


In [25]:
X_scaled

[array([[ 0.08110819, -0.39365545,  0.00506308, ...,  0.22648009,
          0.37594319,  0.37594319],
        [ 0.28971612, -0.73897699, -0.02052879, ...,  0.22648009,
          0.37594319,  0.37594319],
        [ 0.15234016, -1.0049679 , -0.0045368 , ..., -1.8061204 ,
         -2.65997639, -2.65997639],
        ...,
        [-0.88561152,  0.38565126,  0.13392396, ...,  0.74914879,
          0.37594319,  0.37594319],
        [-0.84999553,  0.32498667,  0.13517618, ...,  0.74914879,
          0.37594319,  0.37594319],
        [-0.80929154,  0.29232111,  0.12391828, ...,  0.74914879,
          0.37594319,  0.37594319]]),
 array([[-0.04609178, -1.66294542,  0.01748753, ..., -1.15955986,
          0.37594319,  0.37594319],
        [-0.05117977, -1.66294542,  0.0189774 , ..., -1.15955986,
          0.37594319,  0.37594319],
        [-0.04609178, -1.67694494,  0.01874419, ..., -1.15955986,
          0.37594319,  0.37594319],
        ...,
        [-1.58266731,  1.14629194,  0.30566019, ..., -

In [21]:
# 特徴量数を取得
num_features = X_train_padded.shape[2]
num_features

23

In [22]:
# 4. モデルの構築
model = Sequential()
model.add(Masking(mask_value=0., input_shape=(max_length, num_features)))
model.add(LSTM(128))
model.add(Dense(num_classes, activation='sigmoid'))

# 5. モデルのコンパイルと学習
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
history = model.fit(
    X_train_padded, y_train,
    epochs=50,
    batch_size=32,
    validation_data=(X_test_padded, y_test)
)

# 6. モデルの評価
loss, accuracy = model.evaluate(X_test_padded, y_test)
print(f'Test Loss: {loss}')
print(f'Test Accuracy: {accuracy}')

# 7. 予測と結果の表示
y_pred = model.predict(X_test_padded)
y_pred_classes = np.argmax(y_pred, axis=1)
y_true_classes = np.argmax(y_test, axis=1)
y_pred_labels = label_encoder.inverse_transform(y_pred_classes)
y_true_labels = label_encoder.inverse_transform(y_true_classes)
print(classification_report(y_true_labels, y_pred_labels))

Epoch 1/50


/Users/hinase/.pyenv/versions/3.9.19/lib/python3.9/site-packages/keras/src/layers/core/masking.py:47: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


27/27 ━━━━━━━━━━━━━━━━━━━━ 2s 59ms/step - accuracy: 0.6352 - loss: 0.6363 - val_accuracy: 0.7732 - val_loss: 0.4890
Epoch 2/50
27/27 ━━━━━━━━━━━━━━━━━━━━ 1s 55ms/step - accuracy: 0.8185 - loss: 0.4237 - val_accuracy: 0.8033 - val_loss: 0.4063
Epoch 3/50
27/27 ━━━━━━━━━━━━━━━━━━━━ 2s 57ms/step - accuracy: 0.8734 - loss: 0.2999 - val_accuracy: 0.8279 - val_loss: 0.4126
Epoch 4/50
27/27 ━━━━━━━━━━━━━━━━━━━━ 2s 57ms/step - accuracy: 0.8952 - loss: 0.2279 - val_accuracy: 0.8525 - val_loss: 0.3285
Epoch 5/50
27/27 ━━━━━━━━━━━━━━━━━━━━ 2s 58ms/step - accuracy: 0.9453 - loss: 0.1488 - val_accuracy: 0.8607 - val_loss: 0.3131
Epoch 6/50
27/27 ━━━━━━━━━━━━━━━━━━━━ 2s 61ms/step - accuracy: 0.9521 - loss: 0.1249 - val_accuracy: 0.8798 - val_loss: 0.3255
Epoch 7/50
27/27 ━━━━━━━━━━━━━━━━━━━━ 2s 59ms/step - accuracy: 0.9655 - loss: 0.1221 - val_accuracy: 0.8470 - val_loss: 0.4151
Epoch 8/50
27/27 ━━━━━━━━━━━━━━━━━━━━ 2s 57ms/step - accuracy: 0.9616 - loss: 0.1156 - val_accuracy: 0.8716 - val_loss: 0.

In [ ]:
input_shape = (max_length, num_features)
inputs = Input(shape=input_shape)
x = Masking(mask_value=0.)(inputs)

# num_featuresを定義（入力データの特徴量の次元数）
num_features = input_shape[1]

# Transformerブロックの定義
def transformer_block(x, num_heads, key_dim, ff_dim, rate=0.1):
    # マルチヘッド注意機構
    attn_output = MultiHeadAttention(num_heads=num_heads, key_dim=key_dim)(x, x)
    attn_output = Dropout(rate)(attn_output)

    # 出力次元をnum_featuresに変換
    attn_output = Dense(num_features)(attn_output)

    out1 = Add()([x, attn_output])
    out1 = LayerNormalization(epsilon=1e-6)(out1)

    # フィードフォワードネットワーク
    ffn_output = Dense(ff_dim, activation='relu')(out1)

    # 出力次元をnum_featuresに変換
    ffn_output = Dense(num_features)(ffn_output)

    ffn_output = Dropout(rate)(ffn_output)
    out2 = Add()([out1, ffn_output])
    out2 = LayerNormalization(epsilon=1e-6)(out2)
    return out2

# Transformerブロックの適用
x = transformer_block(x, num_heads=4, key_dim=64, ff_dim=128)

# プーリングと出力層
x = GlobalAveragePooling1D()(x)
x = Dropout(0.1)(x)
outputs = Dense(num_classes, activation='sigmoid')(x)

# モデルの作成
model = Model(inputs=inputs, outputs=outputs)

In [ ]:
# 5. モデルのコンパイルと学習
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
history = model.fit(
    X_train_padded, y_train,
    epochs=50,
    batch_size=32,
    validation_data=(X_test_padded, y_test)
)


In [ ]:
# 6. モデルの評価
loss, accuracy = model.evaluate(X_test_padded, y_test)
print(f'Test Loss: {loss}')
print(f'Test Accuracy: {accuracy}')

# 7. 予測と結果の表示
y_pred = model.predict(X_test_padded)
y_pred_classes = np.argmax(y_pred, axis=1)
y_true_classes = np.argmax(y_test, axis=1)
y_pred_labels = label_encoder.inverse_transform(y_pred_classes)
y_true_labels = label_encoder.inverse_transform(y_true_classes)
print(classification_report(y_true_labels, y_pred_labels))

In [ ]:
import numpy as np
from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt

# 8. 特徴量の重要度評価（Permutation Feature Importance）

# 8.1 ベースラインの性能を計算
# テストデータでの予測
y_pred = model.predict(X_test_padded)
y_pred_classes = np.argmax(y_pred, axis=1)
y_true_classes = np.argmax(y_test, axis=1)

# ベースラインのAccuracyを計算
baseline_accuracy = accuracy_score(y_true_classes, y_pred_classes)
print(f'Baseline Accuracy: {baseline_accuracy:.4f}')

# 8.2 特徴量ごとの重要度を計算
num_features = X_test_padded.shape[2]
feature_importances = []

for feature_idx in range(num_features):
    # テストデータをコピー
    X_test_permuted = X_test_padded.copy()

    # 特徴量をランダムにシャッフル（サンプル間でシャッフル）
    permuted_feature = X_test_permuted[:, :, feature_idx].reshape(X_test_permuted.shape[0], -1)
    np.random.shuffle(permuted_feature)
    X_test_permuted[:, :, feature_idx] = permuted_feature.reshape(X_test_permuted.shape[0], X_test_permuted.shape[1])

    # シャッフル後のデータで予測
    y_pred_permuted = model.predict(X_test_permuted)
    y_pred_classes_permuted = np.argmax(y_pred_permuted, axis=1)

    # シャッフル後のAccuracyを計算
    permuted_accuracy = accuracy_score(y_true_classes, y_pred_classes_permuted)

    # 性能の差を計算（重要度）
    importance = baseline_accuracy - permuted_accuracy
    feature_importances.append(importance)
    print(f'Feature {feature_idx + 1} Importance: {importance:.4f}')

# 8.3 特徴量重要度の可視化
feature_names = [f'Feature {i+1}' for i in range(num_features)]

plt.figure(figsize=(10, 6))
plt.barh(X[7].columns, feature_importances)
plt.xlabel('Decrease in Accuracy')
plt.title('Permutation Feature Importance')
plt.gca().invert_yaxis()
plt.show()


In [ ]:
#  pickleファイルを読み込む
with open('/Users/hinase/Downloads/Th-s/d_nakazawa_acc_check_segments4.7new.pkl', 'rb') as f:
    data = pickle.load(f)

segx = data['d_nakazawa_check_segx']
segy = data['d_nakazawa_check_segy']
segz = data['d_nakazawa_check_segz']

In [ ]:
# Create a simple sine wave as an example of a waveform or signal
x = np.linspace(0, 10, 1000)
y = np.sin(x)

# Define the sections to be colored
color_sections = [(200, 300, 'red'), (450, 550, 'blue'), (700, 800, 'red')]

# Plot the entire waveform in gray
plt.figure(figsize=(10, 2))
plt.plot(x, y, color='gray', linewidth=3)

# Highlight the specified sections with the chosen colors
for start, end, color in color_sections:
    plt.plot(x[start:end], y[start:end], color=color, linewidth=5)

# Add an arrow to indicate the flow of data
# plt.annotate('', xy=(10, 0), xytext=(0, 0),
#              arrowprops=dict(facecolor='black', shrink=0.05, width=2))
plt.xticks(color="None")
plt.yticks(color="None")
plt.tick_params(length=0)
# Remove axes for a cleaner look
plt.axis('off')
#plt.savefig("/Users/hinase/Downloads/plt.svg")

# Show plot
plt.show()

In [ ]:
import cairosvg

# SVGファイルのパス
input_svg_path = "/Users/hinase/Downloads/ss.svg"
# 出力するEPSファイルのパス
output_eps_path = "/Users/hinase/Downloads/ss.eps"

# SVGをEPSに変換
cairosvg.svg2eps(url=input_svg_path, write_to=output_eps_path)

print(f"SVG画像がEPS形式で '{output_eps_path}' に保存されました。")


In [ ]:
import os
import csv
import openpyxl
from natsort import natsorted

folder_path = '/Users/hinase/Downloads/folder3'
folder_2_path = '/Users/hinase/Downloads/folder4'

# Get Excel file names in natural order, filtering only supported Excel files
supported_extensions = ('.xlsx', '.xlsm', '.xltx', '.xltm')
files = natsorted([f for f in os.listdir(folder_path) if f.endswith(supported_extensions)])

# Loop over filenames and build absolute path for each file
for filename in files:
    filepath = os.path.join(folder_path, filename)

    # Access xlsx file and get the first sheet object
    wb = openpyxl.load_workbook(filepath)
    ws_name = wb.sheetnames[0]
    ws = wb[ws_name]

    # Convert to csv and save to folder4
    savecsv_path = os.path.join(folder_2_path, filename.rstrip(".xlsx") + ".csv")
    with open(savecsv_path, 'w', newline="") as csvfile:
        writer = csv.writer(csvfile)
        for row in ws.rows:
            writer.writerow([cell.value for cell in row])
